"""
This is our oroject mainfile
"""

Modules of python
-----------------

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from random import randint

Modules with own classes
------------------------

In [43]:
from glassdata import GlassData
from network import NeuralNetwork

---------------------------------------
Data-set on rho and ANN on molar volume
---------------------------------------

In [44]:
# Dataset of rho
filedbrho='DataBase/rho20oxides.csv'
dbrho=GlassData(filedbrho)
dbrho.info()
dbrho.bounds()

**************
Finished loading dataset
Nb of samples: 64421
Nb of components: 20
**************
Finished loading dataset
Nb of samples: 64421
Nb of components: 20


In [4]:
# Determination of the molar volume
dbrho.oxidemolarmass()
dbrho.molarmass()
dbrho.y=dbrho.MolarM/dbrho.y
dbrho.normalize_y()

In [5]:
# Loading of the ANN model
arch=[20,20,20]
nnmolvol=NeuralNetwork(dbrho.noxide,arch,'gelu','linear')
nnmolvol.compile(3.e-4)
nnmolvol.ArchName(arch)
nnmolvol.load('Models/nnmolarvol'+nnmolvol.namearch+'.h5')
nnmolvol.info()

**************


Model: "sequential_181"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_487 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_488 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_489 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_490 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,283 (5.02 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

------------------------------------------------
Data-set on Young's modulus and ANN on Vt=E/(2G)
------------------------------------------------

In [6]:
filedbE='DataBase/E20oxides.csv'
dbE=GlassData(filedbE)
dbE.info()
dbE.bounds()

**************
Finished loading dataset
Nb of samples: 10846
Nb of components: 20
**************
Finished loading dataset
Nb of samples: 10846
Nb of components: 20


------------------------------
Loading of dissociation energy
------------------------------

In [7]:
datadisso=pd.read_csv('dissociationenergy.csv')
G=np.zeros(dbE.nsample)
for i in range(dbE.nsample):
    G[i]=np.sum(datadisso['G'].values*dbE.x[i,:])
#end for

In [8]:
# Determination of E/G and normalization
dbE.y=dbE.y/(2.*G)
dbE.normalize_y()

------------------------------
Loading of the ANN model on Vt
------------------------------

In [9]:
arch=[20,20,20]
nnmodelEsG=NeuralNetwork(dbE.noxide,arch,'gelu','linear')
nnmodelEsG.compile(1.e-4)
nnmodelEsG.ArchName(arch)
nnmodelEsG.load('Models/nnEsG'+nnmodelEsG.namearch+'.h5')
nnmodelEsG.info()

**************


Model: "sequential_262"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_986 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_987 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_988 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_989 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,283 (5.02 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

---------------------------------------
Data-set on Tannealing=Tg and ANN model
---------------------------------------

In [10]:
# Data-set of Tannealing
filedbTannealing='DataBase/Tannealing20oxides.csv'
dbTannealing=GlassData(filedbTannealing)
dbTannealing.bounds()
dbTannealing.normalize_y()

**************
Finished loading dataset
Nb of samples: 6612
Nb of components: 20


In [11]:
# ANN model on Tannealing
# -----------------------
arch=[20,20,20]
nnTannealing=NeuralNetwork(dbTannealing.noxide,arch,'gelu','linear')
nnTannealing.compile(3.e-4)
nnTannealing.ArchName(arch)
nnTannealing.load('Models/nn'+dbTannealing.nameproperty+nnTannealing.namearch+'.h5')
nnTannealing.info()

**************


Model: "sequential_187"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_585 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_586 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_587 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_588 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,283 (5.02 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Data-set on Tmelt
-----------------

In [12]:
filedbTmelt='DataBase/Tmelt19oxides.csv'
dbTmelt=GlassData(filedbTmelt)
dbTmelt.info()
dbTmelt.bounds()
dbTmelt.normalize_y()

**************
Finished loading dataset
Nb of samples: 5996
Nb of components: 19
**************
Finished loading dataset
Nb of samples: 5996
Nb of components: 19


In [13]:
# ANN model on Tmelt
# ------------------
arch=[20,20,20]
nnTmelt=NeuralNetwork(dbTmelt.noxide,arch,'gelu','linear')
nnTmelt.compile(3.e-4)
nnTmelt.ArchName(arch)
nnTmelt.load('Models/nn'+dbTmelt.nameproperty+nnTmelt.namearch+'.h5')
nnTmelt.info()

**************


Model: "sequential_34"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_116 (Dense)               │ (None, 20)             │           400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_117 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_118 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_119 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,263 (4.94 KB)

 Trainable params: 1,261 (4.93 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

------------------------------
Data-set on Tliq and ANN model
------------------------------

In [14]:
filedbTliq='DataBase\Tsoft20oxides.csv'
dbTliq=GlassData(filedbTliq)
dbTliq.info()
dbTliq.bounds()
dbTliq.normalize_y()

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\laure\AppData\Local\Temp\ipykernel_59752\4115716478.py:1: SyntaxWarning: invalid escape sequence '\T'
  filedbTliq='DataBase\Tsoft20oxides.csv'


**************
Finished loading dataset
Nb of samples: 12531
Nb of components: 20
**************
Finished loading dataset
Nb of samples: 12531
Nb of components: 20


In [15]:
# ANN model on Tliq
# -----------------
arch=[32,32,32,32]
nnTliq=NeuralNetwork(dbTliq.noxide,arch,'gelu','linear')
nnTliq.compile(3.e-4)
nnTliq.ArchName(arch)
#modelfile='Models\nn'+dbTliq.nameproperty+nnTliq.namearch+'.h5'
modelfile='Models/nnTsoft3c20.h5'
nnTliq.load(modelfile)
nnTliq.info()

**************


Model: "sequential_47"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_172 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_173 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_174 (Dense)               │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_175 (Dense)               │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,283 (5.02 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

------------------------------------------
Determination of the bounds for each oxide
------------------------------------------

# Algo genetique

## Variables utiles

In [16]:
rho_db = pd.read_csv(r'DataBase\rho20oxides.csv', header=0)
E_db = pd.read_csv('DataBase\\E20oxides.csv', header=0)
Tg_db = pd.read_csv('DataBase\\Tannealing20oxides.csv', header=0)
Tm_db = pd.read_csv('DataBase\\Tmelt19oxides.csv', header=0)

prop_min = np.array([min(rho_db['rho']),min(E_db['E']),min(Tg_db['Tannealing']),min(Tm_db['Tmelt'])])
prop_max = np.array([max(rho_db['rho']),max(E_db['E']),max(Tg_db['Tannealing']),max(Tm_db['Tmelt'])])

print(prop_min,prop_max)

[1789.99996185   37.49200058  549.73444214  961.68375244] [7072.0000267   110.4        1190.15       2053.15061035]


In [18]:
labels = dbrho.oxide
N_oxides = len(labels)
available_mat = ['SiO2', 'Al2O3', 'MgO', 'CaO', 'Na2O', 'K2O','ZnO', 'TiO2']
prop_label = ['rho','E','Tg','Tmelt']
columns = list(labels)+prop_label+['F']

#Contraintes
xmaxt=np.array([dbrho.xmax,dbE.xmax,dbTannealing.xmax,np.append(dbTmelt.xmax,1.),dbTliq.xmax])
xmax=np.zeros(dbrho.noxide)
for i in range(dbrho.noxide):
    if dbrho.oxide[i] in available_mat:
        xmax[i]=np.min(xmaxt[:,i])

xmin = np.zeros(dbrho.noxide)
xmin[list(dbrho.oxide).index('SiO2')] = 0.5
xmin[list(dbrho.oxide).index('Na2O')] = 0.1

#Parametres fitness function
#L'ordre des parametres est ['rho','E','Tg','Tmelt']
weight=[0,0.35,0,0.65]
minimize=[True,False,False,True]
penalties_onMin = [-np.inf, -np.inf, -np.inf, -np.inf]
penalties_onMax = [np.inf, np.inf, np.inf, np.inf]

N_generations = 2
N_population = 500

survivor_rate = 0.1
parent_rate = 0.5
child_rate = 0.7
mutation_rate = 0
immigration_rate = 1 - survivor_rate - child_rate

strategies = ['China','tournament']
N_tournament = 5
child_strategy = strategies[1]

#population = np.zeros((N_population,len(labels) + len(prop_label) + 1))

N_parents = int(parent_rate * N_population)
N_childs = int(child_rate * N_population)
N_mutants = int(0.1 * N_population)

epsilon = 0.05


## Creation de generations

In [19]:
def prop_calculation(composition):
    rho=dbrho.GlassDensity(nnmolvol,dbrho.oxide,composition)
    E=dbE.YoungModulus(nnmodelEsG,datadisso,dbE.oxide,composition)
    Tg=dbTannealing.physicaly(nnTannealing.model.predict(composition).transpose()[0,:])
    Tmelt=dbTmelt.physicaly(nnTmelt.model.predict(composition[:,:-1]).transpose()[0,:])
    return np.vstack((rho,E,Tg,Tmelt)).transpose()

In [20]:
def normalize(prop):
    #return (prop - prop.min(axis=0))/(prop.max(axis=0)-prop.min(axis=0))
    return (prop - prop_min)/(prop_max - prop_min)

In [21]:
penalties_onMin_normalized = normalize(penalties_onMin)
penalties_onMax_normalized = normalize(penalties_onMax)

In [22]:
def fitness(property_normalized):
    rating = 0
    #print(penalties_onMin_normalized, property_normalized, penalties_onMax_normalized)
    #print(np.all(penalties_onMin_normalized <= property_normalized) and np.all(penalties_onMax_normalized >= property_normalized))
    if np.all(penalties_onMin_normalized <= property_normalized) and np.all(penalties_onMax_normalized >= property_normalized):
        for i in range(len(weight)):
            if minimize[i]:
                rating += (1-property_normalized[i])*weight[i]
            else:
                rating += property_normalized[i]*weight[i]
    return rating

In [23]:
#prop est une array avec les proprietes du verre normalisées, weight est le poids qu'on accorde
#à chacune des proprietes, et minize est une liste de booléens selon qu'on veuille minimiser
#ou maximiser une certaine variable
'''def fitness_func(prop_normalized,weight,minimize):
    rating = np.zeros(prop_normalized.shape[0])
    for i in range(len(weight)):
        if minimize[i]:
            rating += (1-prop_normalized[:,i])*weight[i]
        else:
            rating += prop_normalized[:,i]*weight[i]
    return rating'''
def fitness_func(prop_normalized):
    return np.apply_along_axis(fitness,1,prop_normalized)

In [24]:
# Trie la population par F decroissant et renvoie cette population triée avec une nuovelle colonne qui represente 
# le fitness de chaque composition.
def stack_by_f(population,properties,F):
    population_info = np.column_stack((population,properties,F))
    sorted_arr = population_info[population_info[:, -1].argsort()][::-1]
    return sorted_arr

In [25]:
def init_properties(population):
    prop = prop_calculation(population)
    normalized_prop = normalize(prop)
    F = fitness_func(normalized_prop)
    #print(F)
    sorted_arr = stack_by_f(population, prop, F)
    return sorted_arr

In [26]:
def compute_properties(generation):
    population_sorted = init_properties(generation[:, :20])
    return population_sorted

In [27]:
def init_pop(N_population):
    population,_=dbrho.better_random_composition(N_population,xmin,xmax)
    population = init_properties(population)
    return population

In [35]:
def parent_choice (parents,strategie) :
    if strategie =='China' and N_parents>= 2*N_childs:
        dads = parents[::2]
        moms = parents[1::2]
        return (np.array(moms[:N_childs]),np.array(dads[:N_childs]))    
    if strategie =='tournament':
        dads =[]
        moms =[]
        for i in range(N_childs):
            t = np.array([parents[i,:] for i in np.random.choice(N_parents,N_childs)])
            dad = t[np.argmax(t[:N_tournament,-1])]
            mom = t[np.argmax(t[N_tournament:,-1])]
            dads.append(dad)
            moms.append(mom)
        return (np.array(moms),np.array(dads))
    #Fallback
    t = np.array([parents[i] for i in np.random.choice(N_parents,N_childs*2)])
    return (np.array(t[N_childs:]),np.array(t[:N_childs]))

In [36]:
def population_selection(generation):
    survivors = generation[:int(N_population*survivor_rate)]
    dads,moms = parent_choice(generation[:N_parents],child_strategy)
    #to_be_mutated = sorted_population[int(N_population*survivor_rate):int(N_population*survivor_rate)+int(N_population*mutation_rate)]
    return survivors,dads,moms 

In [37]:
'''def crossover (parents) :
    #Dans parents chaque individu est représenté par 20 premiers floats et le dernier est la valeur de fitness
    childs = np.array([[0.] * len(parents[0])] * N_childs)
    for i in range (N_childs) :
        i1 = randint(0, N_parents-1)
        i2 = randint(0, N_parents-2)
        if i2 == i1 : #problème si deux fois le même parent !!
            i2 += 1
        w1 = parents[i1, -1] / (parents[i1, -1] + parents[i2, -1]) #poids du parent 1
        w2 = parents[i2, -1] / (parents[i1, -1] + parents[i2, -1]) #poids du parent 2
        childs[i] = (w1 * parents[i1] + w2 * parents[i2]) #moyenne pondérée
    return (childs)'''

'def crossover (parents) :\n    #Dans parents chaque individu est représenté par 20 premiers floats et le dernier est la valeur de fitness\n    childs = np.array([[0.] * len(parents[0])] * N_childs)\n    for i in range (N_childs) :\n        i1 = randint(0, N_parents-1)\n        i2 = randint(0, N_parents-2)\n        if i2 == i1 : #problème si deux fois le même parent !!\n            i2 += 1\n        w1 = parents[i1, -1] / (parents[i1, -1] + parents[i2, -1]) #poids du parent 1\n        w2 = parents[i2, -1] / (parents[i1, -1] + parents[i2, -1]) #poids du parent 2\n        childs[i] = (w1 * parents[i1] + w2 * parents[i2]) #moyenne pondérée\n    return (childs)'

In [32]:
# températures de fusion des oxydes (en °C) : 
# [1610, 2045, 2852, 2580]
# dureté des oxydes :
# [7, 9, 5.5]
# TiO2 est un agent nucléant utilisé pour les plaques vitrocéramiques (très faible coeff de dilatation thermique).
# source : L'élémentarium

# fondants = ['Na2O', 'K2O', 'MgO']
# stabilisants = ['CaO', 'ZnO']
# ZnO augmente l'élasticité
# source : https://lasirene.e-monsite.com/pages/le-verre/-.html

# formateurs = ['SiO2']
# SiO2 augmente la dureté du verre
# fondants = ['Na2O', 'K2O', 'MgO']
# stabilisants = ['CaO', 'ZnO']
# source : https://infovitrail.com/contenu.php/fr/d/---la-composition-du-verre/e9b609c9-91f5-4a08-86a6-6112dc12b66d

# fondants = ['Na2O', 'K2O'] + ['CaO', 'MgO'] (mais moins bien que les deux premiers)
# modificateurs = ['CaO', 'MgO', 'Al2O3'] #augmentent les propriétés de durabilité chimique et mécanique (E ??)
# formateurs = ['SiO2'] #essentiels
# source : Franck

fondants = [False, False, False, True, True, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False] # diminuent Tm
durability = [True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False] #augmentent E
other = [False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False]

def crossover (mom, dad) :
    childs = np.zeros((len(mom), len(mom[0])))
    for i in range (N_childs) :
        if mom[i, 21] < dad[i, 21] : #choix du meilleur E
            bestE = dad[i]
        else :
            bestE = mom[i]
        if mom[i, 23] < dad[i, 23] : #choix du meilleur Tm
            bestTm = mom[i]
        else :
            bestTm = dad[i]
        childs[i][fondants] = bestTm[fondants]
        childs[i][durability] = bestE[durability]
        childs[i][other] = (mom[i][other] + dad[i][other])/2
        sum = np.sum(childs[i,1:])
        childs[i,1:] = (1-childs[i,0]) * childs[i,1:] / sum
    return (childs)

'''def crossover (mom, dad) :
    childs = np.zeros((len(mom), len(mom[0])))
    for i in range (N_childs) :
        wMomTm = mom[i, 23] / (mom[i, 23] + dad[i, 23])
        wDadTm = dad[i, 23] / (mom[i, 23] + dad[i, 23])
        wMomE = mom[i, 21] / (mom[i, 21] + dad[i, 21])
        wDadE = dad[i, 21] / (mom[i, 21] + dad[i, 21])
        childs[i][fondants] = wMomTm * mom[i][fondants] + wDadTm * dad[i][fondants]
        childs[i][durability] = wMomE * mom[i][durability] + wDadE * dad[i][durability]
        childs[i][other] = (mom[i][other] + dad[i][other])/2
    return (childs)'''

'def crossover (mom, dad) :\n    childs = np.zeros((len(mom), len(mom[0])))\n    for i in range (N_childs) :\n        wMomTm = mom[i, 23] / (mom[i, 23] + dad[i, 23])\n        wDadTm = dad[i, 23] / (mom[i, 23] + dad[i, 23])\n        wMomE = mom[i, 21] / (mom[i, 21] + dad[i, 21])\n        wDadE = dad[i, 21] / (mom[i, 21] + dad[i, 21])\n        childs[i][fondants] = wMomTm * mom[i][fondants] + wDadTm * dad[i][fondants]\n        childs[i][durability] = wMomE * mom[i][durability] + wDadE * dad[i][durability]\n        childs[i][other] = (mom[i][other] + dad[i][other])/2\n    return (childs)'

In [33]:
def mutation (mutants) :
    #les mutants sont les meilleures compositions entre 40% et 50%
    for j in range (N_mutants) :
        iplus = randint(0, 19) #choix de l'oxyde qui gagne epsilon
        imoins = randint(0, 19) #choix de l'oxyde qui perd epsilon
        if imoins == iplus : #problème si deux fois le même oxyde !!
            imoins = (1 + imoins)%19
        mutantPlus = mutants[j, iplus]
        mutantMoins = mutants[j, imoins]
        if (mutantMoins > epsilon + xmin[j]) and (mutantPlus < xmax[j] - epsilon) :
            mutants[j, iplus] = mutantPlus + epsilon
            mutants[j, imoins] = mutantMoins - epsilon
    return (mutants)

In [38]:
def new_generation(old_generation):
    survivors,dads,moms = population_selection(old_generation)
    child = crossover(dads,moms)
    immigrants = init_pop(N_population - (len(survivors) + len(child)))
    new_population = np.vstack((np.vstack((survivors,child)),immigrants))
    new_population = compute_properties(new_population)
    return new_population

In [ ]:
def evolution(generation,N,graphe = True):
    convergence_graph = []
    for _ in range(N):
        generation = new_generation(generation)
        fit_score = np.mean(generation[:10],axis = 0)[-1]
        convergence_graph.append(fit_score)
    if graphe:
        plt.plot(np.arange(N),convergence_graph)
        plt.title('evolution of the generational fitness')
        
    return generation

In [40]:
initial_pop = init_pop(N_population)
compositions = evolution(initial_pop,N_generations)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 984us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 869us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step


In [41]:
df = pd.DataFrame(compositions,columns=columns)
df.to_csv('generation_final_oui_le_bon_fin_presque.csv')
df
#df.to_csv('generation_final_29_11b.csv')

,SiO2,B2O3,Al2O3,MgO,CaO,BaO,Li2O,Na2O,K2O,ZnO,...,TiO2,GeO2,ZrO2,P2O5,V2O5,rho,E,Tg,Tmelt,F
0,0.500046,0.0,8.493203e-04,0.004082,1.422278e-03,0.0,0.0,0.199113,4.055782e-05,1.147476e-01,...,1.796993e-01,0.0,0.0,0.0,0.0,2933.806309,80.385418,721.921877,1144.531588,0.747022
1,0.500001,0.0,1.618381e-03,0.003388,1.180608e-03,0.0,0.0,0.165280,3.366633e-05,1.919991e-01,...,1.364984e-01,0.0,0.0,0.0,0.0,3016.182532,74.632283,854.407614,1227.956830,0.669721
2,0.500003,0.0,1.915198e-05,0.004541,7.316413e-03,0.0,0.0,0.166496,3.942992e-05,1.845370e-01,...,1.370478e-01,0.0,0.0,0.0,0.0,3009.633750,75.646624,848.717293,1241.702178,0.666405
3,0.500000,0.0,3.564574e-02,0.004880,7.863533e-03,0.0,0.0,0.178947,4.237848e-05,1.423200e-01,...,1.303008e-01,0.0,0.0,0.0,0.0,2907.887704,78.673385,856.182741,1266.930227,0.665911
4,0.500003,0.0,2.189235e-05,0.005190,8.363287e-03,0.0,0.0,0.190320,4.507178e-05,1.694313e-01,...,1.266254e-01,0.0,0.0,0.0,0.0,2969.326660,77.585511,848.664068,1260.096454,0.664758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.772978,0.0,8.681320e-05,0.049127,7.021769e-02,0.0,0.0,0.107582,4.184831e-06,5.879100e-10,...,4.475925e-06,0.0,0.0,0.0,0.0,2438.130608,72.706942,835.901243,1898.455656,0.261177
496,0.797857,0.0,6.082520e-04,0.007072,8.890056e-02,0.0,0.0,0.103452,2.881560e-08,2.109914e-03,...,1.278653e-10,0.0,0.0,0.0,0.0,2438.000302,72.270859,854.902845,1934.363564,0.237699
497,0.735194,0.0,4.086372e-02,0.003297,5.978102e-04,0.0,0.0,0.109041,6.612124e-02,2.839451e-02,...,1.649058e-02,0.0,0.0,0.0,0.0,2498.271508,66.703848,804.474601,1911.519028,0.224579
498,0.824927,0.0,1.720943e-02,0.000338,1.728804e-10,0.0,0.0,0.100016,8.248050e-05,5.742684e-02,...,1.719281e-07,0.0,0.0,0.0,0.0,2458.956495,69.312000,856.078468,2033.033307,0.164735


In [ ]:
df

,SiO2,B2O3,Al2O3,MgO,CaO,BaO,Li2O,Na2O,K2O,ZnO,...,TiO2,GeO2,ZrO2,P2O5,V2O5,rho,E,Tg,Tmelt,F
0,0.500000,0.0,2.016686e-02,0.000515,1.320917e-07,0.0,0.0,0.181001,5.311387e-03,0.122073,...,0.170933,0.0,0.0,0.0,0.0,2918.012746,77.446447,782.663530,1166.322589,0.719936
1,0.500001,0.0,2.304691e-04,0.000001,2.128081e-02,0.0,0.0,0.385901,2.815030e-05,0.003871,...,0.088686,0.0,0.0,0.0,0.0,2697.674891,75.433368,711.154776,1151.031922,0.719378
2,0.500000,0.0,3.069999e-11,0.000001,2.017110e-02,0.0,0.0,0.365777,2.668237e-05,0.006618,...,0.107405,0.0,0.0,0.0,0.0,2720.629472,75.703151,720.248526,1155.809044,0.717828
3,0.500071,0.0,9.976802e-14,0.000001,2.016821e-02,0.0,0.0,0.365725,2.667855e-05,0.006618,...,0.107390,0.0,0.0,0.0,0.0,2720.593250,75.702495,720.263834,1155.846013,0.717803
4,0.508738,0.0,5.429683e-06,0.000188,1.234519e-05,0.0,0.0,0.220696,8.601170e-03,0.106125,...,0.155633,0.0,0.0,0.0,0.0,2889.798209,78.810290,831.714263,1196.723541,0.708378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.796823,0.0,1.789525e-03,0.003639,8.141087e-02,0.0,0.0,0.100932,4.855134e-09,0.014827,...,0.000578,0.0,0.0,0.0,0.0,2458.224773,71.931120,857.212018,1921.180807,0.243919
496,0.559458,0.0,1.975797e-01,0.001071,1.646487e-02,0.0,0.0,0.100009,4.630056e-02,0.079109,...,0.000009,0.0,0.0,0.0,0.0,2611.683250,79.533980,969.118060,1991.528747,0.238523
497,0.635718,0.0,1.377215e-01,0.000716,3.671130e-02,0.0,0.0,0.148983,2.271992e-02,0.003287,...,0.014144,0.0,0.0,0.0,0.0,2492.380289,73.080594,906.142019,1975.343850,0.217182
498,0.808350,0.0,6.035978e-02,0.000061,5.278721e-04,0.0,0.0,0.100468,2.253834e-04,0.029753,...,0.000256,0.0,0.0,0.0,0.0,2417.173905,72.029683,925.100715,1967.204276,0.216984
